<a href="https://colab.research.google.com/github/DAEN690-S22-OA/OceanicAirspaceProject/blob/main/VerticalSeparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Up The Environment

In [1]:
#Installing the required libraries
!pip install geopy
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=ebfc2d1d3baf3aa79cf5320854d029cb0738e5a42c19af986be6279fecb8b09a
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [2]:
#Importing the required libraries
import pandas as pd
import pandasql as ps
# from pyspark.sql import SparkSession
import os
import sys
import re
from datetime import datetime
from geopy.distance import geodesic as GD
from math import radians, cos, sin, asin, sqrt
from google.colab import files
import io
import numpy as np
from shapely.geometry import Point, Polygon

# Importing Raw Data & Pre-Processing

In [ ]:
#Upload the full datafile 
uploadedfile = files.upload('DEC_24_2021.csv')
rawData_df = pd.read_csv(io.BytesIO(uploadedfile['DEC_24_2021.csv']))
print(f'Total record count : ',len(rawData_df.index))

In [ ]:
#New dataframe with selected attributes from the raw data (Flight Level Column added)
airspaceData_df =rawData_df[["FRN73TMRPDateTimeOfMessageRec","FRN131HRPWCFloatingPointLat","FRN131HRPWCFloatingPointLong","FRN145FLFlightLevel",
                 "FRN170TITargetId","RESHSelectedHeading","FRN80TATargetAddress",
                 "FRN161TNTrackNumber"]]

#Rename columns to make it easier to read
airspaceData_df = airspaceData_df.rename(columns={'FRN73TMRPDateTimeOfMessageRec': 'DateTime', 
                                                  'FRN131HRPWCFloatingPointLat': "Latitude", 
                                                  'FRN131HRPWCFloatingPointLong': "Longitude",  
                                                  'FRN145FLFlightLevel':"Height",
                                                  'FRN170TITargetId': "TargetID", 
                                                  'RESHSelectedHeading': "SelectedHeading", 
                                                  'FRN80TATargetAddress': "TargetAddress",
                                                  'FRN161TNTrackNumber': "TrackNumber"})

#Preview to ensure everything is looking as expected 
airspaceData_df.head()

In [ ]:
# Remove anything less than 240 flight level 
airspaceData1 = airspaceData_df[(airspaceData_df['Height'] >= 240)]

#Preview Data
airspaceData1.head()

In [ ]:
# Convert dask dataframe to pandas dataframe
airspaceDF = airspaceData1.compute()

In [ ]:
# Remove unnescessary characters 
char = ['T','Z']
for x in char:
    airspaceDF["DateTime"] = airspaceDF["DateTime"].str.replace( x ," ")

# Formatted Datetime
airspaceDF["DateTime"] = pd.to_datetime(airspaceDF["DateTime"], format="%Y-%m-%d %H:%M:%S")

In [ ]:
# Create 4 new columns for Hour, Minute, Second and Microsecond
airspaceDF["Hour"] = airspaceDF["DateTime"].dt.hour
airspaceDF["Minute"] = airspaceDF["DateTime"].dt.minute
airspaceDF["Second"] = airspaceDF["DateTime"].dt.second
airspaceDF["microSecond"] = airspaceDF["DateTime"].dt.microsecond

In [ ]:
# Reorder columns
airspaceDF = airspaceDF[["DateTime","Hour","Minute","Second","microSecond","Latitude","Longitude","Height",
                                   "TargetID","SelectedHeading","TargetAddress",
                                   "TrackNumber"]]

# Preview Dataframe
airspaceDF.head()

In [ ]:
airspaceData = airspaceData[(airspaceData['Second'] < 5)]
print(f'Total record count : ',len(airspaceData.index))

## Upload datafile here during testing

In [3]:
#---------------REMOVE AFTER FINAL TESTING --------------------------------

#Upload the data file after pre-processing (only the selected attribues and only above level 240)
uploadedfile = files.upload()
airspaceDF = pd.read_csv(io.BytesIO(uploadedfile['airspaceData.csv']))

Saving airspaceData.csv to airspaceData.csv


In [4]:
airspaceDF.head(5)

,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,2021-12-24 12:36:02.773,12,36,2,773000,37.307144,-139.574170,350.00,ASA899,NaN,A7B779,1024.0
1,2021-12-24 12:36:02.750,12,36,2,750000,48.981194,-131.626113,389.75,JAL10,70.31250,86E84C,624.0
2,2021-12-24 12:36:02.695,12,36,2,695000,44.079895,-167.602190,330.00,CKS258,NaN,AA7BD6,1024.0
3,2021-12-24 12:36:02.617,12,36,2,617000,3.637665,-174.884747,379.75,QFA76,206.71875,7C8068,3616.0
4,2021-12-24 12:36:02.563,12,36,2,563000,30.918227,-129.410266,389.75,AAL675,57.65625,AB5C12,1024.0


In [5]:
#Some column names may be imported with leading spaces. This will remove that
airspaceDF.columns = [x.strip() for x in airspaceDF.columns]

print(airspaceDF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211679 entries, 0 to 211678
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   DateTime         211679 non-null  object 
 1   Hour             211679 non-null  int64  
 2   Minute           211679 non-null  int64  
 3   Second           211679 non-null  int64  
 4   microSecond      211679 non-null  int64  
 5   Latitude         211679 non-null  float64
 6   Longitude        211679 non-null  float64
 7   Height           211679 non-null  float64
 8   TargetID         211648 non-null  object 
 9   SelectedHeading  162560 non-null  float64
 10  TargetAddress    211679 non-null  object 
 11  TrackNumber      211441 non-null  float64
dtypes: float64(5), int64(4), object(3)
memory usage: 19.4+ MB
None


In [6]:
# Rename Height to FlightLevel
airspaceDF.rename(columns={'Height': 'FlightLevel'}, inplace=True)

In [7]:
# Change Flight Level Unit * 100
airspaceDF['FlightLevel'] = airspaceDF['FlightLevel'].apply(lambda x: x * 100)

In [8]:
# Replace missing value with -1
airspaceDF['SelectedHeading'] = airspaceDF['SelectedHeading'].fillna(-1)

In [9]:
# Assign Direction "E" for 0-180 degree, "W" for 180-360 degree, "NA" is record with null values 
conditionlist = [
    (airspaceDF['SelectedHeading'] < 0) ,
    (airspaceDF['SelectedHeading'] >= 0) & (airspaceDF['SelectedHeading'] <180),
    (airspaceDF['SelectedHeading'] > 180)]
choicelist = ['NA', 'E', 'W']
airspaceDF['Direction'] = np.select(conditionlist, choicelist)

airspaceDF.head(5)

,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,FlightLevel,TargetID,SelectedHeading,TargetAddress,TrackNumber,Direction
0,2021-12-24 12:36:02.773,12,36,2,773000,37.307144,-139.574170,35000.0,ASA899,-1.00000,A7B779,1024.0,NA
1,2021-12-24 12:36:02.750,12,36,2,750000,48.981194,-131.626113,38975.0,JAL10,70.31250,86E84C,624.0,E
2,2021-12-24 12:36:02.695,12,36,2,695000,44.079895,-167.602190,33000.0,CKS258,-1.00000,AA7BD6,1024.0,NA
3,2021-12-24 12:36:02.617,12,36,2,617000,3.637665,-174.884747,37975.0,QFA76,206.71875,7C8068,3616.0,W
4,2021-12-24 12:36:02.563,12,36,2,563000,30.918227,-129.410266,38975.0,AAL675,57.65625,AB5C12,1024.0,E


#Create the polygon for Hawaii airspace and remove any points that fall within that polygon

In [10]:
#Coordinates for Hawaii airspace
v0 = (26.14472222, -158.62194444) 
v1 = (26.105, -160.63166667)
v2 = (25.67611111, -161.69111111)
v3 = (25.05666667, -162.64972222)
v4 = (24.16889, -163.26638889)
v5 = (23.25833, -163.855)
v6 = (22.20555556, -163.91444444)
v7 = (33.10266389, 130.47177778)
v8 = (20.11666667, -163.3)
v9 = (19.65805556,-162.69944444)
v10 = (19.415, -162.38361111)
v11 = (18.40777778, -160.81416667)
v12 = (18.0525, -160.26972222)
v13 = (17.75583333, -159.53888889)
v14 = (17.17055556, -157.75666667) 
v15 = (17.805,-156.06805556)
v16 = (18.10888889, -155.71166667)
v17 = (19.14222222, -154.48333333)
v18 = (19.22293333, -151.87963333)
v19 = (20.69694444, -151.01916667) 
v20 = (21.54777778, -151.46638889)
v21 = (22.34416667,-151.88527778)
v22 = (23.02416667, -152.57777778)
v23 = (23.78055556, -153.36611111)
v24 = (24.29583333, -154.25)
v25 = (24.72138889, -155.26305556)
v26 = (25.19583333, -156.42111111)

# Polygon
coords = [v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16, v17, v18, v19, v20, v21, v22, v23, v24, v25, v26]
poly = Polygon(coords)

In [11]:
hawaiiAir = []
loc = 0

while loc < len(airspaceDF):
  p1 = Point(airspaceDF.iloc[loc][5], airspaceDF.iloc[loc][6])
  hawaiiAir.append(p1.within(poly))
  loc = loc + 1

airspaceDF['nearHawaii'] = hawaiiAir

airspaceDF.head(10)

,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,FlightLevel,TargetID,SelectedHeading,TargetAddress,TrackNumber,Direction,nearHawaii
0,2021-12-24 12:36:02.773,12,36,2,773000,37.307144,-139.574170,35000.0,ASA899,-1.000000,A7B779,1024.0,NA,False
1,2021-12-24 12:36:02.750,12,36,2,750000,48.981194,-131.626113,38975.0,JAL10,70.312500,86E84C,624.0,E,False
2,2021-12-24 12:36:02.695,12,36,2,695000,44.079895,-167.602190,33000.0,CKS258,-1.000000,AA7BD6,1024.0,NA,False
3,2021-12-24 12:36:02.617,12,36,2,617000,3.637665,-174.884747,37975.0,QFA76,206.718750,7C8068,3616.0,W,False
4,2021-12-24 12:36:02.563,12,36,2,563000,30.918227,-129.410266,38975.0,AAL675,57.656250,AB5C12,1024.0,E,False
5,2021-12-24 12:36:02.547,12,36,2,547000,38.192945,-142.006113,37000.0,ACA034,21.796875,C01732,1321.0,E,False
6,2021-12-24 12:36:02.523,12,36,2,523000,23.224319,-141.326627,39000.0,DAL40,23.203125,A64CDB,1024.0,E,False
7,2021-12-24 12:36:02.516,12,36,2,516000,13.020080,156.802084,38975.0,ANZ1090,158.906250,C820CF,1024.0,E,False
8,2021-12-24 12:36:02.516,12,36,2,516000,3.513344,-155.194150,39000.0,ANZ1008,21.093750,C820CE,145.0,E,False
9,2021-12-24 12:36:02.484,12,36,2,484000,19.601937,167.751742,35000.0,MGE305,-1.000000,A9228A,1024.0,NA,False


In [12]:
airspaceDF = airspaceDF[(airspaceDF['nearHawaii'] == False)]
airspaceDF = airspaceDF.drop(columns=['nearHawaii'])

airspaceDF.head(10)

,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,FlightLevel,TargetID,SelectedHeading,TargetAddress,TrackNumber,Direction
0,2021-12-24 12:36:02.773,12,36,2,773000,37.307144,-139.574170,35000.0,ASA899,-1.000000,A7B779,1024.0,NA
1,2021-12-24 12:36:02.750,12,36,2,750000,48.981194,-131.626113,38975.0,JAL10,70.312500,86E84C,624.0,E
2,2021-12-24 12:36:02.695,12,36,2,695000,44.079895,-167.602190,33000.0,CKS258,-1.000000,AA7BD6,1024.0,NA
3,2021-12-24 12:36:02.617,12,36,2,617000,3.637665,-174.884747,37975.0,QFA76,206.718750,7C8068,3616.0,W
4,2021-12-24 12:36:02.563,12,36,2,563000,30.918227,-129.410266,38975.0,AAL675,57.656250,AB5C12,1024.0,E
5,2021-12-24 12:36:02.547,12,36,2,547000,38.192945,-142.006113,37000.0,ACA034,21.796875,C01732,1321.0,E
6,2021-12-24 12:36:02.523,12,36,2,523000,23.224319,-141.326627,39000.0,DAL40,23.203125,A64CDB,1024.0,E
7,2021-12-24 12:36:02.516,12,36,2,516000,13.020080,156.802084,38975.0,ANZ1090,158.906250,C820CF,1024.0,E
8,2021-12-24 12:36:02.516,12,36,2,516000,3.513344,-155.194150,39000.0,ANZ1008,21.093750,C820CE,145.0,E
9,2021-12-24 12:36:02.484,12,36,2,484000,19.601937,167.751742,35000.0,MGE305,-1.000000,A9228A,1024.0,NA


# Filter A Time Block

In [13]:
def minuteFilter(HourCounter,MinuteCounter):

  global airspaceDF

  #create SQL query for flights between the start and end time
  sql1 = "SELECT *, min(Second) FROM airspaceDF WHERE Hour = '{0}' and Minute = '{1}' GROUP BY TargetID ORDER BY TargetID, Second".format(HourCounter, MinuteCounter)

  #Run query and store results
  recordsInMinute = ps.sqldf(sql1, globals())
  del recordsInMinute['min(Second)']

  return (recordsInMinute)

# Check For Proximity

In [14]:
def lonTableCreate(recordsByMinuteDF):
  #Create tables to check for proximity in latitude
  sql4 = "SELECT * FROM recordsByMinuteDF ORDER BY Longitude"

  #Run query and store results
  LongitudeOrderDF = ps.sqldf(sql4, locals())

  return (LongitudeOrderDF)

In [15]:
def latTableCreate(recordsByMinuteDF):
  #Create tables to check for proximity in latitude
  sql3 = "SELECT * FROM recordsByMinuteDF ORDER BY Latitude"

  #Run query and store results
  LatitudeOrderDF = ps.sqldf(sql3, locals())

  return (LatitudeOrderDF)

In [16]:
# Implement the formula below
def distance_d(point0,pointX):
# The function "radians" is found in the math module
 LoA = radians(point0[1])  
 LoB = radians(pointX[1])
 LaA=  radians(point0[0])  
 LaB = radians(pointX[0]) 
# The "Haversine formula" is used.
 D_Lo = LoB - LoA 
 D_La = LaB - LaA 
 P = sin(D_La / 2)**2 + cos(LaA) * cos(LaB) * sin(D_Lo / 2)**2  
   
 Q = 2 * asin(sqrt(P))   
 # The earth's radius in kilometers.
 R_km = 6371  
 
 # Change the kilometer to  nautical miles
 R_nm = R_km*0.539956803

# Then we'll compute the outcome.
 return(Q * R_nm)

In [17]:
# Create function to set up boundary within 25 nm by latitude, longitude 
def limit_lon(point0):
  '''
  use with LongitudeOrderDF
  note: distance from point to longitude boundary of each row is around 24.9715
  '''
  LaA = radians(point0[0])
  onedeg_long = cos(LaA)*(69.172*0.868976242)
  add = 25/onedeg_long 
  pointlimit = (point0[0],point0[1]+add)
  return pointlimit[1]

def limit_lat(point0):
  '''
  use with LatitudeOrderDF
  note: distance from point to longitude boundary of each row is 25.016857125339488
  '''
  onedeg_lat = 60 
  add = 25/onedeg_lat
  pointlimit = (point0[0]+add,point0[1])
  return pointlimit[0]

In [19]:
# Create function to select, merge and add the values from analyzing Longitude and Latitude
def newDF(OrderDF,x,y,d):
    """DF is Long/LatitudeOrderDF
       x = long/latpoint_a
       y = long/latpoint_b
       d = long/latdistance_ab"""
    # select rows that index is in list 'point_a', 'point_b'
    A = OrderDF.loc[x,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','FlightLevel',
                             'TargetID', 'SelectedHeading', 'TargetAddress']]
    B = OrderDF.loc[y,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','FlightLevel',
                             'TargetID', 'SelectedHeading', 'TargetAddress']]
    # Join 2 tables by the "TargetID" of point a (for the uniquness)
    OrderResult = pd.merge(A.reset_index(drop=True),B.reset_index(drop=True),left_index=True, right_index=True)
    # add distance column
    OrderResult['Distance'] = d
    return OrderResult

In [31]:
#Calculate the distance of the points closest to each other by longitidue and latitude
def proximityCalc(LongitudeOrderDF, LatitudeOrderDF):
  longpoint_a = []
  longpoint_b = []
  longdistance_ab = []

  latpoint_a = []
  latpoint_b = []
  latdistance_ab = []

  for a in LongitudeOrderDF.index:
    for n in range(1,len(LongitudeOrderDF)):
      b = a+n
      if b < len(LongitudeOrderDF):
        point0 = LongitudeOrderDF.loc[a,'Latitude'], LongitudeOrderDF.loc[a,'Longitude']
        pointX = LongitudeOrderDF.loc[b,'Latitude'], LongitudeOrderDF.loc[b,'Longitude']
        if pointX[1] <= limit_lon(point0): # Check if longitude of pointX is within the boundary
          distance = distance_d(point0,pointX)
          if distance <= 25: # Check distance within 25 nm
            longpoint_a.append(a)
            longpoint_b.append(b)
            longdistance_ab.append(distance)
        else:
          break

  for a in LatitudeOrderDF.index:   
    for n in range(1,len(LatitudeOrderDF)):
      b = a+n
      if b < len(LatitudeOrderDF):
        point0 = LatitudeOrderDF.loc[a,'Latitude'], LatitudeOrderDF.loc[a,'Longitude']
        pointX = LatitudeOrderDF.loc[b,'Latitude'], LatitudeOrderDF.loc[b,'Longitude']    
        if pointX[0] <= limit_lat(point0):# Check if latitude of pointX is within the boundary
          distance = distance_d(point0,pointX)
          if distance <= 25: # Check distance within 25 nm
            latpoint_a.append(a)
            latpoint_b.append(b)
            latdistance_ab.append(distance)
        else:
          break

  # Apply function to select and merge data frame
  LongOrderResult = newDF(LongitudeOrderDF,longpoint_a, longpoint_b,longdistance_ab)
  LatOrderResult = newDF(LatitudeOrderDF,latpoint_a, latpoint_b,latdistance_ab)

  # Concatenate results from order by longitude and latitude
  Resultsdf = pd.concat([LongOrderResult,LatOrderResult]).reset_index(drop=True)

  # Delete duplicate pairs of TargetID x and y regardless of order
  Resultsdf['list_target'] = Resultsdf.apply(lambda row: tuple(sorted([row['TargetID_x']]+[row['TargetID_y']])), axis = 1)
  ResultsDF = Resultsdf.drop_duplicates(subset = 'list_target',keep = 'first').reset_index(drop = True)
  ResultsDF.drop('list_target', axis=1, inplace=True)

  return (ResultsDF)
  

# Calculate Height Differences

In [21]:
def distanceCalc(resultsDF):
  heightDifference = []
  potentialLoss1000 = []
  potentialLoss400 = []

  counter = 0

  while counter < len(resultsDF):
    difference = abs((resultsDF['FlightLevel_x'][counter]) - (resultsDF['FlightLevel_y'][counter]))
    heightDifference.append(difference)

    if difference <= 1000:
      potentialLoss1000.append('True')
      if difference <= 400:
        potentialLoss400.append('True')
      else:
        potentialLoss400.append('False')
    else:
      potentialLoss1000.append('False')
      potentialLoss400.append('False')

    counter = counter + 1

  resultsDF['HeightDifference_ft'] = heightDifference
  resultsDF['potentialLoss400'] = potentialLoss400
  resultsDF['potentialLoss1000'] = potentialLoss1000

  return (resultsDF)


# Call the functions

In [30]:
HourCounter = 00
finalResults = pd.DataFrame()

while HourCounter < 1:
  #Create table for the minute
  for MinuteCounter in range(0,60):
    #Create table for the minute
    recordsByMinuteDF = minuteFilter(HourCounter,MinuteCounter)
    
    #Create longitude and latitude table for proximity analysis
    LongitudeOrderDF = lonTableCreate(recordsByMinuteDF)
    LatitudeOrderDF = latTableCreate(recordsByMinuteDF)
    
    #calculate proximity
    resultsDF = proximityCalc(LongitudeOrderDF, LatitudeOrderDF)
    if resultsDF.empty == True:
      # if the results dataframe is empty, then break out of for-loop
      break
    else:
      #Calculate distance
      resultsDF = distanceCalc(resultsDF)
      #Add the results for this minute to the overall results 
      finalResults = pd.concat([finalResults, resultsDF], ignore_index=True)
  HourCounter = HourCounter + 1

In [25]:
finalResults.to_csv('results1Hr_new.csv', encoding = 'utf-8-sig') 
files.download('results1Hr_new.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Print Results

In [26]:
#Get the first entry for this minute of time
sql6 = "SELECT * FROM finalResults WHERE potentialLoss400 = 'True' "

#Run query and store results
LossCandidates400 = ps.sqldf(sql6, locals())

In [27]:
print(LossCandidates400)

                DateTime_x  Hour_x  Minute_x  Second_x  microSecond_x  \
0  2021-12-24 00:24:00.531       0        24         0         531000   
1  2021-12-24 00:45:00.875       0        45         0         875000   

   Latitude_x  Longitude_x  FlightLevel_x TargetID_x  SelectedHeading_x  ...  \
0   22.298813  -155.816895        30000.0     DAL495          45.703125  ...   
1   21.387222  -155.755821        30575.0    SWA1385         196.875000  ...   

  Latitude_y Longitude_y  FlightLevel_y  TargetID_y  SelectedHeading_y  \
0  22.278488 -155.472823        29800.0      AAL432          78.046875   
1  21.663071 -155.441362        30475.0     SWA1310          68.203125   

   TargetAddress_y   Distance  HeightDifference_ft  potentialLoss400  \
0           AB271F  19.153653                200.0              True   
1           ABFD8C  24.140765                100.0              True   

  potentialLoss1000  
0              True  
1              True  

[2 rows x 26 columns]


In [28]:
#Get the first entry for this minute of time
sql7 = "SELECT * FROM finalResults WHERE potentialLoss1000 = 'True' "

#Run query and store results
LossCandidates1000 = ps.sqldf(sql7, locals())

In [29]:
print(LossCandidates1000)

                 DateTime_x  Hour_x  Minute_x  Second_x  microSecond_x  \
0   2021-12-24 00:00:01.406       0         0         1         406000   
1   2021-12-24 00:00:00.531       0         0         0         531000   
2   2021-12-24 00:01:00.289       0         1         0         289000   
3   2021-12-24 00:04:00.398       0         4         0         398000   
4   2021-12-24 00:05:03.188       0         5         3         188000   
5   2021-12-24 00:07:01.711       0         7         1         711000   
6   2021-12-24 00:19:03.875       0        19         3         875000   
7   2021-12-24 00:19:02.594       0        19         2         594000   
8   2021-12-24 00:20:04.773       0        20         4         773000   
9   2021-12-24 00:20:04.773       0        20         4         773000   
10  2021-12-24 00:21:03.711       0        21         3         711000   
11  2021-12-24 00:22:00.820       0        22         0         820000   
12  2021-12-24 00:23:03.055       0   